In [16]:
import selenium
from selenium.webdriver import ActionChains

In [17]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import os
import bs4
import time
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import numpy as np

In [18]:
options = Options()
options.add_argument("start-maximized")#fullscreen mode
# options.add_argument("--headless")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options)
def patching_get(driver, url):
    # Run this until works
    done = False
    tries = 10
    x = 0
    while not done:
        x += 1
        if x > tries:
            raise
        try:
            driver.get(url)
            done = True
        except KeyboardInterrupt:
            raise
        except: 
            time.sleep(1)
    return driver

In [27]:
allCategories = []
def getAllCategories(driver,base_url = "https://www.flipkart.com/"):
    driver = patching_get(driver,base_url)
    menPath = driver.find_element_by_xpath('//*[@id="container"]/div/div[2]/div/ul/li[3]/span').text
    womenPath = driver.find_element_by_xpath('//*[@id="container"]/div/div[2]/div/ul/li[4]/span').text
    print(menPath)
    print(womenPath)
    
    links = driver.find_elements_by_class_name('_3GtRpC')
    print(len(links))
    for i in range(len(links)):
        if(i == 0 or i == 1):
            continue
        if(i > 3):
            break
        majCatName = ""
        if(i == 2):
            majCatName = menPath
        elif(i == 3):
            majCatName = womenPath
        print(links[i])
        el = links[i]
        lowCategories = el.find_elements_by_tag_name('a')
        print(len(lowCategories))
        tot = []
        allLinks = []
        for lowCat in lowCategories:
            singleLink = []
            name = lowCat.get_attribute('title')
            
            link = lowCat.get_attribute('href')
            singleLink.append(name)
            singleLink.append(link)
            allLinks.append(singleLink)
        tot.append(majCatName)
        tot.append(allLinks)
        allCategories.append(tot)
    driver.quit()    
    return allCategories     
        
        
    print(len(links))

In [28]:
getAllCategories(driver)

Men
Women
7
<selenium.webdriver.remote.webelement.WebElement (session="366546fb2c1eabb3c9277347a46355ec", element="6ad7b97c-f813-4cbf-a60c-8c262d1bc766")>
82
<selenium.webdriver.remote.webelement.WebElement (session="366546fb2c1eabb3c9277347a46355ec", element="0588058c-7025-4081-8266-4dfe00cef91d")>
90


[['Men',
  [['Footwear',
    'https://www.flipkart.com/mens-footwear/pr?sid=osp,cil&otracker=nmenu_sub_Men_0_Footwear'],
   ['Sports Shoes',
    'https://www.flipkart.com/mens-footwear/sports-shoes/pr?sid=osp,cil,1cu&otracker=nmenu_sub_Men_0_Sports%20Shoes'],
   ['Casual Shoes',
    'https://www.flipkart.com/mens-footwear/casual-shoes/pr?sid=osp,cil,e1f&otracker=nmenu_sub_Men_0_Casual%20Shoes'],
   ['Formal Shoes',
    'https://www.flipkart.com/mens-footwear/formal-shoes/pr?sid=osp,cil,ssb&otracker=nmenu_sub_Men_0_Formal%20Shoes'],
   ['Sandals & Floaters',
    'https://www.flipkart.com/mens-footwear/sandals-floaters/pr?sid=osp,cil,e83&otracker=nmenu_sub_Men_0_Sandals%20%26%20Floaters'],
   ['Flip- Flops',
    'https://www.flipkart.com/mens-footwear/slippers-flip-flops/pr?sid=osp,cil,e1r&otracker=nmenu_sub_Men_0_Flip-%20Flops'],
   ['Loafers',
    'https://www.flipkart.com/mens-footwear/casual-shoes/loafers~type/pr?sid=osp%2Ccil%2Ce1f&otracker=nmenu_sub_Men_0_Loafers'],
   ['Boots',
  